<!--
Copyright (c) 2025 Milin Patel
Hochschule Kempten - University of Applied Sciences

Autonomous Driving: AI Safety and Security Workshop
This project is licensed under the MIT License.
See LICENSE file in the root directory for full license text.
-->

*Copyright © 2025 Milin Patel. All Rights Reserved.*

# Notebook 19: V2X Communication and Cooperative Perception

**Session 5: Standards Integration and Deployment**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_5_Advanced_Topics/notebooks/19_V2X_Communication.ipynb)

**Author:** Milin Patel  
**Institution:** Hochschule Kempten

---

## Learning Objectives

After completing this notebook, you will understand:
- What V2X (Vehicle-to-Everything) communication is and why it matters
- Different types of V2X: V2V, V2I, V2P, V2N
- C-V2X vs DSRC technology comparison
- Cooperative perception: seeing beyond line-of-sight
- V2X security threats and ISO/SAE 21434 application
- Real-world deployment challenges

---

## Setup and Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
print("✓ Libraries imported successfully")

---

## 1. What is V2X Communication?

**V2X (Vehicle-to-Everything)** allows vehicles to communicate with:
- **V2V**: Other vehicles
- **V2I**: Infrastructure (traffic lights, road signs)
- **V2P**: Pedestrians (smartphones)
- **V2N**: Network (cloud services)

### Why V2X Matters

Autonomous vehicles have physical limitations:
- **Line-of-sight**: Cannot see around corners or through obstacles
- **Sensor range**: LiDAR effective to ~200m, radar ~250m
- **Occlusions**: Trucks, buildings block view
- **Weather**: Fog, rain reduce visibility

**V2X solves this** by sharing information wirelessly:
- Vehicle behind truck sees what's ahead via V2V
- Traffic light tells approaching vehicle it will turn red in 5 seconds
- Emergency vehicle broadcasts its location before it's visible
- Pedestrian's phone warns vehicle of crossing intent

### Safety Benefits

Studies show V2X can prevent:
- **80% of intersection crashes** (through V2I traffic light warnings)
- **70% of rear-end collisions** (through V2V emergency braking alerts)
- **50% of pedestrian accidents** (through V2P warnings)

**Key Insight:** V2X is complementary to sensors, not a replacement.

---

## 2. V2X Technologies: C-V2X vs DSRC

Two competing standards exist:

### DSRC (Dedicated Short Range Communications)
- Based on IEEE 802.11p (WiFi variant)
- Operating frequency: 5.9 GHz
- Range: ~300m
- Latency: ~100ms
- **Status**: Being phased out in US and EU

### C-V2X (Cellular V2X)
- Based on 3GPP standards (4G LTE, 5G NR)
- Operating frequency: 5.9 GHz (same as DSRC)
- Range: ~1000m (5G)
- Latency: ~20ms (5G)
- **Status**: Industry preference, China deployment

### Comparison Table

In [ ]:
comparison = pd.DataFrame({
    'Feature': ['Technology Base', 'Range', 'Latency', 'Bandwidth', 
                'Infrastructure', 'Deployment Status', 'Industry Support'],
    'DSRC (802.11p)': ['WiFi', '~300m', '~100ms', '6-27 Mbps', 
                        'Roadside units', 'Legacy', 'Declining'],
    'C-V2X (4G/5G)': ['Cellular', '~1000m', '~20ms', '100+ Mbps', 
                       'Cell towers + RSU', 'Growing', 'Strong']
})

display(comparison)

print("\n📊 Current Trend: C-V2X is winning globally")
print("   - China: Mandating C-V2X in new vehicles")
print("   - EU: Moving toward C-V2X")
print("   - US: FCC reallocated 5.9 GHz band away from DSRC")

---

## 3. Cooperative Perception

**Cooperative Perception** = Vehicles share what they "see" with each other

### How It Works

1. **Vehicle A** detects object with camera/LiDAR
2. **Vehicle A** broadcasts object data via V2V:
   - Object type (car, pedestrian, cyclist)
   - Position (GPS coordinates)
   - Velocity vector
   - Timestamp
3. **Vehicle B** (behind truck, cannot see object) receives message
4. **Vehicle B** incorporates object into its world model
5. **Vehicle B** can now plan around invisible object

### Benefits

- **See through occlusions**: Know what's behind trucks, buildings
- **Extended range**: Perception beyond sensor limits
- **Redundancy**: Cross-validate own detections with others
- **Early warnings**: React to hazards before they're visible

### Challenges

- **Coordinate systems**: Must align different vehicle reference frames
- **Time synchronization**: GPS time accurate to ~10ns needed
- **Data association**: Which message corresponds to which object?
- **Trust**: How to handle false or malicious messages?
- **Bandwidth**: Limited spectrum, cannot share everything

In [ ]:
# Simulate cooperative perception scenario
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))

# Scenario: Vehicle behind truck
# Without V2X
ax1.set_title('Without V2X: Cannot See Obstacle', fontsize=14, fontweight='bold')
ax1.set_xlim(0, 100)
ax1.set_ylim(0, 20)
ax1.axis('off')

# Ego vehicle
ax1.add_patch(plt.Rectangle((10, 8), 8, 4, color='blue', label='Ego Vehicle'))
ax1.text(14, 10, 'You', ha='center', va='center', color='white', fontweight='bold')

# Truck (occlusion)
ax1.add_patch(plt.Rectangle((40, 7), 15, 6, color='gray', label='Truck (blocks view)'))
ax1.text(47.5, 10, 'Truck', ha='center', va='center', color='white', fontweight='bold')

# Hidden obstacle
ax1.add_patch(plt.Rectangle((70, 8.5), 6, 3, color='red', alpha=0.3))
ax1.text(73, 10, '?', ha='center', va='center', color='red', fontsize=20, fontweight='bold')
ax1.text(73, 5, 'Obstacle\n(hidden)', ha='center', fontsize=9, color='red')

# Sensor range
ax1.plot([18, 40], [10, 10], 'b--', linewidth=2, label='Sensor blocked')
ax1.text(29, 12, 'Cannot see!', ha='center', fontsize=10, color='red', fontweight='bold')

# With V2X
ax2.set_title('With V2X: Receives Warning via V2V', fontsize=14, fontweight='bold')
ax2.set_xlim(0, 100)
ax2.set_ylim(0, 20)
ax2.axis('off')

# Ego vehicle
ax2.add_patch(plt.Rectangle((10, 8), 8, 4, color='blue'))
ax2.text(14, 10, 'You', ha='center', va='center', color='white', fontweight='bold')

# Truck
ax2.add_patch(plt.Rectangle((40, 7), 15, 6, color='gray'))
ax2.text(47.5, 10, 'Truck', ha='center', va='center', color='white', fontweight='bold')

# Obstacle (now known via V2V)
ax2.add_patch(plt.Rectangle((70, 8.5), 6, 3, color='red'))
ax2.text(73, 10, '⚠', ha='center', va='center', color='white', fontsize=16, fontweight='bold')
ax2.text(73, 5, 'Obstacle\n(known via V2V)', ha='center', fontsize=9, color='red')

# V2V communication
ax2.annotate('', xy=(18, 15), xytext=(70, 15),
            arrowprops=dict(arrowstyle='<->', color='green', lw=2))
ax2.text(44, 16.5, 'V2V Message\n(obstacle position)', ha='center', 
        fontsize=9, color='green', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n✓ Cooperative Perception allows 'seeing' through obstacles")

---

## 4. V2X Message Types

V2X uses standardized messages defined in **SAE J2735**:

### Basic Safety Message (BSM)
Broadcast by every vehicle 10 times per second:
- Vehicle position (GPS)
- Speed and heading
- Acceleration
- Brake status
- Vehicle size

### Cooperative Awareness Message (CAM)
European equivalent of BSM

### Signal Phase and Timing (SPaT)
Broadcast by traffic lights:
- Current phase (red, yellow, green)
- Time remaining in phase
- Upcoming phase changes

### Road Side Alert (RSA)
Warnings from infrastructure:
- Construction zones
- Icy roads
- Emergency vehicle approaching

### Personal Safety Message (PSM)
From pedestrian smartphones:
- Pedestrian location
- Movement direction
- Crossing intent

---

## 5. V2X Security (ISO/SAE 21434)

V2X introduces new cybersecurity threats:

### Threat 1: Message Spoofing
**Attack:** Attacker broadcasts false BSM claiming emergency vehicle approaching
**Impact:** Vehicles unnecessarily pull over, cause congestion
**Mitigation:** Digital signatures, Public Key Infrastructure (PKI)

### Threat 2: Sybil Attack
**Attack:** One attacker creates multiple fake vehicle identities
**Impact:** False congestion reports, denial of service
**Mitigation:** Certificate management, position verification

### Threat 3: Replay Attack
**Attack:** Record and re-broadcast old messages
**Impact:** Outdated information causes wrong decisions
**Mitigation:** Timestamps, sequence numbers

### Threat 4: Jamming
**Attack:** Overwhelm 5.9 GHz frequency with noise
**Impact:** V2X communication fails
**Mitigation:** Frequency hopping, fallback to onboard sensors

### Threat 5: Privacy
**Issue:** Continuous BSM broadcast allows vehicle tracking
**Mitigation:** Pseudonymous certificates, change ID frequently

### Security Architecture (ISO/SAE 21434)

V2X security uses:
1. **PKI**: Certificate Authority issues certificates
2. **Digital Signatures**: Every message signed
3. **Pseudonyms**: Temporary IDs changed every 5-10 minutes
4. **Misbehavior Detection**: Reject inconsistent messages
5. **Hardware Security Module (HSM)**: Secure key storage

In [ ]:
# V2X Security Threats Summary
threats = pd.DataFrame({
    'Threat': ['Message Spoofing', 'Sybil Attack', 'Replay Attack', 
               'Jamming', 'Privacy Tracking'],
    'Attack Method': [
        'Send false emergency vehicle BSM',
        'Create 100 fake vehicle IDs',
        'Re-broadcast old position',
        'Transmit noise on 5.9 GHz',
        'Track vehicle by BSM ID'
    ],
    'Impact': [
        'Vehicles pull over unnecessarily',
        'False congestion, DoS',
        'Outdated info causes crashes',
        'V2X completely disabled',
        'User location revealed'
    ],
    'Mitigation': [
        'Digital signatures + PKI',
        'Position verification',
        'Timestamps, nonces',
        'Sensor fallback, frequency hopping',
        'Pseudonyms changed every 5 min'
    ],
    'ISO/SAE 21434 CAL': ['CAL-3', 'CAL-4', 'CAL-2', 'CAL-3', 'CAL-2']
})

display(threats)

print("\n⚠ V2X security is CRITICAL for safety")
print("   A single spoofed message can cause crashes")

---

## 6. Real-World Deployment Challenges

Despite benefits, V2X deployment is slow:

### Challenge 1: Infrastructure Cost
- Need roadside units (RSU) at every intersection: $10K-$30K each
- Major cities have 1000s of intersections
- Total cost: **billions of dollars**
- Who pays? Government budgets limited

### Challenge 2: Chicken-and-Egg Problem
- Drivers won't buy V2X-equipped cars without infrastructure
- Governments won't build infrastructure without V2X cars
- **Solution:** Mandate V2X in new vehicles (China approach)

### Challenge 3: Technology Transition
- DSRC vs C-V2X debate delayed deployment by 10 years
- Early DSRC investments wasted as industry shifts to C-V2X

### Challenge 4: Interoperability
- Different standards in US, EU, China
- Cross-border vehicles need multiple radios

### Challenge 5: Cybersecurity Complexity
- PKI infrastructure expensive to operate
- Certificate management across millions of vehicles
- Privacy vs security trade-offs

### Where V2X IS Deployed

**China:**
- Shanghai: 3000+ RSUs installed
- Beijing: V2X corridors operational
- Mandate: All new vehicles by 2025

**Europe:**
- Germany: A9 digital test track
- Netherlands: Truck platooning

**United States:**
- Limited pilot projects (Tampa, NYC, Wyoming)
- No federal mandate
- Industry-led deployments by GM, Ford

---

## 7. Key Takeaways

1. **V2X is essential for safe autonomous driving** - sensors alone have fundamental limitations

2. **C-V2X is winning** over DSRC globally due to better performance and 5G integration

3. **Cooperative perception** extends situational awareness beyond line-of-sight

4. **Security is critical** - a single spoofed message can cause crashes (ISO/SAE 21434 applies)

5. **Deployment is slow** due to infrastructure costs and chicken-and-egg problem

6. **China is leading** with government mandates and massive infrastructure investment

7. **V2X complements sensors** but does not replace them - both are needed

---

## Further Reading

- [SAE J2735: V2X Communications Message Set Dictionary](https://www.sae.org/standards/content/j2735_202007/)
- [3GPP C-V2X Specifications](https://www.3gpp.org/)
- [ETSI ITS Standards (Europe)](https://www.etsi.org/technologies/intelligent-transport)
- [5GAA: 5G Automotive Association](https://5gaa.org/)
- Chen et al. (2020): "Cooperative Perception for Autonomous Driving: A Survey"

---

*Copyright © 2025 Milin Patel. All Rights Reserved.*